https://quant.stackexchange.com/questions/69355/mean-variance-optimization-max-sharpe-ratio-portfolio

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import yfinance as yf
import re 
import decimal as dec

In [25]:
sp500=pd.read_excel("SP500-CapWeight-Ref.xlsx")
symbols = sp500["Symbol"]
symbols = symbols[0:20]

In [26]:
query = str( symbols.to_list() )
query = re.sub(r"[\[\]',]",'', query)
print(query)

data = yf.download(query, period="3mo");

MSFT AAPL NVDA AMZN GOOGL META GOOG BRK-B TSLA AVGO LLY JPM UNH V JNJ XOM MA HD PG COST


c:\Users\ljohny\miniconda3\envs\value\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]c:\Users\ljohny\miniconda3\envs\value\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]c:\Users\ljohny\miniconda3\envs\value\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*******               15%%                      ] 

In [27]:
df_prices = data['Adj Close']
print(df_prices.iloc[0:5,0:5])

Ticker            AAPL        AMZN        AVGO       BRK-B        COST
Date                                                                  
2023-10-25  170.874893  121.389999  845.741089  336.899994  536.776001
2023-10-26  166.670425  119.570000  822.965820  336.160004  534.443420
2023-10-27  167.998672  127.739998  834.522705  331.709991  529.983276
2023-10-30  170.065933  132.710007  837.399475  337.410004  541.548523
2023-10-31  170.545319  133.089996  837.518921  341.329987  539.167114


In [28]:
df_returns = df_prices.pct_change(1)
print(df_returns.iloc[0:5,0:5])

Ticker          AAPL      AMZN      AVGO     BRK-B      COST
Date                                                        
2023-10-25       NaN       NaN       NaN       NaN       NaN
2023-10-26 -0.024606 -0.014993 -0.026929 -0.002196 -0.004346
2023-10-27  0.007969  0.068328  0.014043 -0.013238 -0.008345
2023-10-30  0.012305  0.038907  0.003447  0.017184  0.021822
2023-10-31  0.002819  0.002863  0.000143  0.011618 -0.004397


In [29]:
returns = df_returns.iloc[1:].values
print(returns[0])

[-0.02460553 -0.014993   -0.02692936 -0.00219647 -0.00434554 -0.0254993
 -0.02651064 -0.01042966 -0.01695592  0.00256406 -0.03130654 -0.05622428
 -0.03732512 -0.03751433 -0.03477822 -0.00531211 -0.031353   -0.00348926
 -0.02351699 -0.00911683]


In [30]:
def mean_cov_returns(returns):
    mean_returns = np.mean(returns, axis=0)
    cov_matrix = np.cov(returns.T)
    return mean_returns, cov_matrix

mean_returns, cov_matrix = mean_cov_returns(returns)

_, num_assets = df_returns.shape
print(f'{num_assets=}')

print(f'{mean_returns.shape=}, {cov_matrix.shape=}')

num_assets=20
mean_returns.shape=(20,), cov_matrix.shape=(20, 20)


In [31]:
def calculate_portfolio_stats(weights, mean_returns, cov_matrix):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_std_dev
   

In [32]:
# Equality constraints
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

            # {'type': 'eq', 'fun': lambda weights: calculate_portfolio_stats(weights, mean_returns, cov_matrix)[0] - mean_returns}

print(f'{constraints=}')

# LB and UB
bounds = tuple((0,1) for asset in range(num_assets))
print(f'{bounds=}')

# Initial Weights
initial_weights = np.ones(num_assets) / num_assets
print(f'{initial_weights=}')

constraints={'type': 'eq', 'fun': <function <lambda> at 0x000001A8223E7600>}
bounds=((0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1))
initial_weights=array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])


In [33]:
# check
portfolio_return, portfolio_std_dev = calculate_portfolio_stats(initial_weights, mean_returns, cov_matrix)
print(f'{portfolio_return=}, {portfolio_std_dev=}') 

portfolio_return=0.0024841857691053925, portfolio_std_dev=0.007136923376601573


In [34]:
def objective_fn(weights, mean_returns, cov_matrix):
    portfolio_return, portfolio_std_dev = calculate_portfolio_stats(
        weights, mean_returns, cov_matrix)
    return portfolio_std_dev

In [35]:
# Optimize using scipy's minimize function
result = minimize(objective_fn, initial_weights, (mean_returns, cov_matrix),
                  method='SLSQP', bounds=bounds, constraints=constraints)

In [36]:
weights = result.x
print(weights)

[5.31286084e-02 4.80131803e-02 0.00000000e+00 8.28762673e-02
 5.74036207e-02 4.75708368e-02 5.45856618e-02 2.24218638e-02
 1.03935690e-01 1.16810353e-01 0.00000000e+00 2.62919027e-18
 5.38659793e-03 1.89735380e-19 0.00000000e+00 5.81846380e-02
 1.19262239e-18 1.90839273e-01 5.20716420e-02 1.06771767e-01]


In [37]:
weights_list = list( weights )
weights_list = [ dec.Decimal(w) for w in weights_list]
print(weights_list)


[Decimal('0.053128608435350520300399779216604656539857387542724609375'), Decimal('0.048013180281452925990404168032910092733800411224365234375'), Decimal('0'), Decimal('0.08287626727740608767458496686231228522956371307373046875'), Decimal('0.057403620655443986231514230667016818188130855560302734375'), Decimal('0.0475708367886636118715415477709029801189899444580078125'), Decimal('0.0545856617853156256270352741921669803559780120849609375'), Decimal('0.0224218637866386606705138007100686081685125827789306640625'), Decimal('0.10393568996757342193237860783483483828604221343994140625'), Decimal('0.11681035339916075177502108317639795131981372833251953125'), Decimal('0'), Decimal('2.62919026827734825246807304210960865020751953125E-18'), Decimal('0.005386597928688633719895850759940003626979887485504150390625'), Decimal('1.8973538018496327595130424015223979949951171875E-19'), Decimal('0'), Decimal('0.058184638032221729087023476267859223298728466033935546875'), Decimal('1.19262238973405487740819808

In [38]:
np.sum(weights_list)

Decimal('1.000000000000000022226144536')

In [39]:
portfolio_return, portfolio_std_dev = calculate_portfolio_stats(initial_weights, mean_returns, cov_matrix)
print(f'{portfolio_return=}, {portfolio_std_dev=}') 

portfolio_return=0.0024841857691053925, portfolio_std_dev=0.007136923376601573


In [45]:
portfolio_return, portfolio_std_dev = calculate_portfolio_stats(weights, mean_returns, cov_matrix)
print(f'{(portfolio_return)=}, {portfolio_std_dev=}') 
annualized_return = (((1 + portfolio_return)**365) - 1) * 100
print(f'{annualized_return}')

(portfolio_return)=0.0014549766529842797, portfolio_std_dev=0.005238539871223547
70.00888391419491


9